In [1]:
import json
import re
import os
from graphviz import Graph


### Listing the top 10 files in the directory tables

In [11]:
dirpath = os.path.join( os.getcwd() , "tables")
filepath = os.path.join(dirpath,'report.mlb_orgs.json')



In [17]:
grandTreeList = []
def constructTreeList(filepath):
    branches = createBranch(filepath) 
    for key,tables in branches.items():
        for table in tables:
            constructTreeList(getFilePath(table))
    grandTreeList.append(branches)
constructTreeList(filepath)
for item in grandTreeList:
    print(item)


{'broadcast.delivered_games': []}
{'report.delivered_new_pk': ['broadcast.delivered_games']}
{'base.games': []}
{'rundown.access_rules_end': []}
{'broadcast.priority_lists': []}
{'report.exp_access_rules_hist': ['rundown.access_rules_end', 'broadcast.priority_lists']}
{'report.delivered_exceptions_hist': ['report.delivered_new_pk', 'base.games', 'report.exp_access_rules_hist']}


In [19]:
class ConstructGrantTree:
    def __init__(self,grandTreeList):
        self.treeDict = {}
        self.branches = []
        self.leaves = [] 
        self.allNodes = []
        self.treeList = grandTreeList
        self.parent_child_list=[]
        self.rootNode = None
        self.removeLeafNodes()
        self.getParentChildList()       
     
    def getParentChildList(self):
        for node in self.allNodes:
            self.parent_child_list.append(self.getMyParent(node))
        self.getRootNode()

                    
    def getRootNode(self):
        for parent,item in self.parent_child_list:
            if parent == None:
                self.rootNode = item
    
    def removeLeafNodes(self):
        for dictionary in self.treeList:
            for key,values in dictionary.items():
                if values:
                    self.branches.append(key)
                    self.allNodes.append(key)
                else :
                    self.leaves.append(key)
                    self.allNodes.append(key)
                    
    def getMyParent(self,node):
        parent = None
        for dictionary in self.treeList:
            for key,values in dictionary.items():
                if node in values:
                    parent = key
        return (parent,node)
    
tree = ConstructGrantTree(grandTreeList)
print(tree.parent_child_list)


[('report.delivered_new_pk', 'broadcast.delivered_games'), ('report.delivered_exceptions_hist', 'report.delivered_new_pk'), ('report.delivered_exceptions_hist', 'base.games'), ('report.exp_access_rules_hist', 'rundown.access_rules_end'), ('report.exp_access_rules_hist', 'broadcast.priority_lists'), ('report.delivered_exceptions_hist', 'report.exp_access_rules_hist'), (None, 'report.delivered_exceptions_hist')]


In [ ]:
allNodes